# Ініціалізація проекту

In [ ]:
!pip install urllib
!pip install requests

In [8]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/Study/DevNet/HikkaShop")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Модуль обробки природної мови (NLP)

## Використання Waifu AI Api

In [10]:
import requests as re
import urllib.parse
import Keys

def get_response(from_name, to_name, situation, user_id, message):
  data_params = {
    "from_name": from_name,
    "to_name": to_name,
    "situation": situation,
    "message": message,

    "user_id": user_id,
    "preset_mode": "waifu",
    "translate_from": "uk",
  }

  def strTransform(params):
      res = ""
      for param, value in params.items():
          res += param+"="+urllib.parse.quote(value)+"&"
      return res[:len(res)-1]

  params = {
      "url": "https://waifu-ai.p.rapidapi.com/path",
      "headers": {
          'content-type': 'application/x-www-form-urlencoded',
          'x-rapidapi-host': 'waifu-ai.p.rapidapi.com',
          'x-rapidapi-key': Keys.keyMax
      },
      "data": strTransform(data_params)
  }
  
  response = re.post(**params)
  return response.text

## Створення логфайлу бесіди

In [ ]:
def logOutput(log):
  from datetime import datetime
  now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
  with open("logfiles/"+now+".txt", "a", encoding="UTF-8") as ouf:
    ouf.write(log)

## Генерація user_id

In [ ]:
import string
import random
alnum = string.ascii_letters + string.digits
user_id = ''.join(random.choice(alnum) for i in range(64))
print(user_id)

'N2UkvFxvvq7Rb0RzZKmeAGsIi0x1zAUfic1UleLNpcSCFZLAbXIDxS9DBXRccevN'

## Вхідні дані розмови

In [17]:
from merch_list import merch_list

cur_merch_list = merch_list

def get_stock(cur_merch_list):
  stock = ""
  for merch, v in cur_merch_list.items():
    qnty = v[0]
    price = v[1]
    stock += f". {to_name} може продати {qnty} штук {merch} за ціною {price} гривень"
  return stock

def update_stock(cur_merch_list, cash, sold_merch):
  merch
  cur_merch_list[sold_merch][0]
  new_cash = 
  return 


In [14]:
#@title { run: "auto", form-width: "80%" }
from_name = "Тарас-кун" #@param {type: "raw"}
to_name =   "Акено-тян" #@param {type: "raw"}
situation = "Акено-тян продає аніме-товари в магазині." #@param {type: "raw"}
situation += get_stock()

## Надсилання повідомлень

In [ ]:
log = f"""from_name = {from_name}
to_name   = {to_name}
situation = {situation}
user_id   = {user_id}
---\n
"""

while(True):
  message = input(f"{from_name}: ")
  log += f"{from_name}: {message}\n"
  response = get_response(
    from_name = from_name,
    to_name   = to_name,
    situation = situation,
    user_id   = user_id,
    message   = message
  )

  print(f'{to_name}: {response}')
  log += f'{to_name}: {response}\n'

  if (message == "Пока!"):
    logOutput(log)
    break

Акено-тян: Вау! Хто ти?! Ви більше не повинні показувати тут своє обличчя!!
Акено-тян: Так, привіт!
Акено-тян: Пам&#39;ятати тебе? У жодному разі. Це було три роки тому чи щось таке.
Акено-тян: Гм, дайте мені побачити... це не було нічого важливого...
Акено-тян: так. Чому?
Акено-тян: Добре, добре. Гм, чому б тобі не прийти сюди і не дізнатися?
